# Block Creation based on number of points within input PC

Description coming...

## Imports Functions & Variables for this notebook

Description coming soon...

In [2]:
# For paths
import os 

# For data processing
import torch
import numpy as np
from torch.utils.data import Dataset
import math


# For 3D visualization
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

# Setting the root for this repo
ROOT = os.path.realpath("..")

# Creating/ Making the point cloud
def makePC(point_data, color_data=np.array([])):
    pcd = o3d.geometry.PointCloud() #Create PC object
    pcd.points = o3d.utility.Vector3dVector(point_data) #Give coordinates
    #Coloring the PC
    if len(color_data) == 0:
        pcd.paint_uniform_color([1, 0, 0])
    else:
        pcd.colors = o3d.utility.Vector3dVector(color_data)
    return pcd

# Axis alignment of given point cloud
def axisAlignment(main_pcd):
    """Rotating/axis aligning the pointcloud
    input: main_pcd = initial pointcloud (open3d.geometry.PointCloud object)
    output: main_pcd = rotated pointcloud (open3d.geometry.PointCloud object)"""
    bbox = main_pcd.get_oriented_bounding_box()
    inverse_R = np.linalg.inv(bbox.R)
    main_pcd.rotate(inverse_R)
    return main_pcd

## Loading of the input data

In [18]:
# Path creation for easier loading
input_pc_path = ROOT + "/data/testdata/data_labelled_int.npy"

# Loading relevant data
input_pc = np.load(input_pc_path) #ndarray
input_pc_points, input_pc_labels = input_pc[:, 0:6], input_pc[:, 6]

# Print checks for the input data
print("--------- Print checks for input data ---------")
print(f"The input data consists of {len(input_pc)} points.")
print(f"The input data is of shape: {input_pc.shape}")
print(f"The input data contains {np.count_nonzero(input_pc_labels)} fracture points.")
print(f"Therefore, {len(input_pc) - np.count_nonzero(input_pc_labels)} points are non-fracture points in the entire sample.")
print(f"This means that {np.round(np.count_nonzero(input_pc_labels) / len(input_pc) * 100, 4)}% of all points are fracture points")
print("------------------------------------------------")

--------- Print checks for input data ---------
The input data consists of 662525 points.
The input data is of shape: (662525, 7)
The input data contains 8199 fracture points.
Therefore, 654326 points are non-fracture points in the entire sample.
This means that 1.2375% of all points are fracture points
------------------------------------------------


### Analysis of the input cloud coordinate spread


In [19]:
print('---------- Print Check for X-Coordinate ----------')
print(f"The minimum value in x-direction is: {np.min(input_pc_points[:, 0])}")
print(f"The maximum value in x-direction is: {np.max(input_pc_points[:, 0])}")
print(f'The mean value in x-direction is: {np.mean(input_pc_points[:, 0])}')
print(f'The variance in x-direction is: {np.var(input_pc_points[:, 0])}')
print('---------- Print Check for Y-Coordinate ----------')
print(f"The minimum value in y-direction is: {np.min(input_pc_points[:, 1])}")
print(f"The maximum value in y-direction is: {np.max(input_pc_points[:, 1])}")
print(f'The mean value in y-direction is: {np.mean(input_pc_points[:, 1])}')
print(f'The variance in y-direction is: {np.var(input_pc_points[:, 1])}')
print('---------- Print Check for Z-Coordinate ----------')
print(f"The minimum value in z-direction is: {np.min(input_pc_points[:, 2])}")
print(f"The maximum value in z-direction is: {np.max(input_pc_points[:, 2])}")
print(f'The mean value in z-direction is: {np.mean(input_pc_points[:, 2])}')
print(f'The variance in z-direction is: {np.var(input_pc_points[:, 2])}')

---------- Print Check for X-Coordinate ----------
The minimum value in x-direction is: 0.0
The maximum value in x-direction is: 288.8758689999813
The mean value in x-direction is: 165.61342139521435
The variance in x-direction is: 4299.517704445124
---------- Print Check for Y-Coordinate ----------
The minimum value in y-direction is: 0.0
The maximnum value in y-direction is: 226.1995279993862
The mean value in y-direction is: 104.99673473649423
The variance in y-direction is: 2716.543347393639
---------- Print Check for Z-Coordinate ----------
The minimum value in z-direction is: 0.0
The maximnum value in z-direction is: 177.267736
The mean value in z-direction is: 74.85754577858197
The variance in z-direction is: 2083.001901645439


## Block center coordinate creation with half block padding around edges

Description coming soon...

In [5]:
# Given HP and parameters
num_point = 4096
block_size = 10.0
sample_rate = 1.0

# Determining maximum number of blocks for sample
max_num_blocks = int(np.sum(input_pc_labels.size) * sample_rate / num_point)

# Determining maximum and minimum points
coord_min, coord_max = np.amin(input_pc_points, axis=0)[:3], np.max(input_pc_points, axis=0)[:3]

# Creating spacing variables for block center creation
blocks_x_res = math.floor(np.sqrt(max_num_blocks))
blocks_y_res = math.floor(np.sqrt(max_num_blocks))
num_blocks = (blocks_x_res - 1)**2 #actual no. of blocks after padding with half on each side
blocks_center_x_start = (1 / blocks_x_res) * coord_max[0]
blocks_center_y_start = (1 / blocks_y_res) * coord_max[1]

# Creating block center coordinates
blocks_center_x_coords = np.linspace(start=blocks_center_x_start, 
                                     stop=coord_max[0]-blocks_center_x_start, 
                                     num=blocks_x_res-1)
blocks_center_y_coords = np.linspace(start=blocks_center_y_start, 
                                     stop=coord_max[1]-blocks_center_y_start, 
                                     num=blocks_y_res-1)
blocks_center_xx_coords, blocks_center_yy_coords = np.meshgrid(blocks_center_x_coords, 
                                                                blocks_center_y_coords)
#TODO: Calculate the average z-coordinate for each block and append as block center
blocks_center_zz_coords = np.zeros((blocks_center_xx_coords.shape))
blocks_center_coords = np.hstack([blocks_center_xx_coords.reshape((-1, 1)),
                                  blocks_center_yy_coords.reshape((-1, 1)),
                                  blocks_center_zz_coords.reshape((-1,1))])

## Analysis of block creation based on block size

Description coming...

In [7]:
# Placeholder variables to fill with values 
block_point_idxs = [] #actual indices of points of input pc within respective block
block_num_frac_points = [] #actual labels of points of input pc within respective block
block_num_input_pc_points = [] #number of points of input pc in respective block
block_mins = [] #block minimum coordinate
block_maxs = [] #block maximum coordiante


# Looping through the created block centers to check for number of points
for index, block_center in enumerate(blocks_center_coords):
    block_min = block_center[:2] - [block_size / 2.0, block_size / 2.0]
    block_max = block_center[:2] + [block_size / 2.0, block_size / 2.0]
    # Determining the number of points in the block
    points_in_block = np.where((input_pc_points[:, 0] >= block_min[0]) & \
                               (input_pc_points[:, 0] <= block_max[0]) & \
                               (input_pc_points[:, 1] >= block_min[1]) & \
                               (input_pc_points[:, 1] <= block_max[1]))[0]
    #Pulling corresponding values for each block for analysis
    selected_points_idxs = input_pc_points[points_in_block]
    block_point_idxs.append(selected_points_idxs)
    selected_points_labels = input_pc_labels[points_in_block]
    block_num_frac_points.append(np.sum(selected_points_labels))
    block_num_input_pc_points.append(points_in_block.size)
    block_mins.append(block_min)
    block_maxs.append(block_max)


# Preparation for the print checks of the input pc
points_in_blocks_with_enough_points = [i for i in block_num_input_pc_points if i >= num_point]
idxs_blocks_with_enough_points = np.array([idx for idx, val in enumerate(block_num_input_pc_points) if val >= num_point], dtype=int)
points_in_blocks_with_fewer_points = [i for i in block_num_input_pc_points if i < num_point]
idxs_blocks_with_fewer_points = np.array([idx for idx, val in enumerate(block_num_input_pc_points) if val < num_point], dtype=int)
num_blocks_without_any_points = [i for i in block_num_input_pc_points if i == 0]
indices_of_blocks_with_zero_points = [idx for idx, val in enumerate(block_num_input_pc_points) if val == 0]
num_blocks_with_too_little_points = num_blocks - len(points_in_blocks_with_enough_points)
general_block_frac_point_avg = np.round(np.mean(block_num_frac_points), 2)
blocks_with_enough_points_frac_point_avg = np.round(np.mean(np.array(block_num_frac_points)[idxs_blocks_with_enough_points]), 2)
blocks_with_fewer_points_frac_point_avg = np.round(np.mean(np.array(block_num_frac_points)[idxs_blocks_with_fewer_points]), 2)


# Print checks for the blocks within the input pc
print('---------- Print checks for the input pc blocks ----------')
print(f'The input point cloud was divided into {len(blocks_center_coords)} blocks.')
print(f'The number of points of the original input pc that are now contained within blocks is: {np.sum(block_num_input_pc_points)}')
print(f'The number of blocks that have more points than the subsampling goal: {len(points_in_blocks_with_enough_points)}')
print(f'The number of blocks that have less points then the subsampling goal: {num_blocks_with_too_little_points}')
print(f'The number of blocks which have no subsampled points at all: {len(indices_of_blocks_with_zero_points)}')
print(f'The maximum amount of points within a block is: {np.max(points_in_blocks_with_enough_points)}')
print(f'The average amount of points of the blocks that have enough points is: {np.round(np.mean(points_in_blocks_with_enough_points), 2)}')
print(f'The average amount of points of the blocks that have fewer points is: {np.round(np.mean(points_in_blocks_with_fewer_points), 2)}')
print(f'On average there are {general_block_frac_point_avg} fracture points within a block.')
print(f'On average the blocks with enough points have {blocks_with_enough_points_frac_point_avg} fracture points within a block.')
print(f'On average the blocks with fewer points have {blocks_with_fewer_points_frac_point_avg} fracture points within a block.')
print(f'The amount of blocks that do not contain any fracture points are: {block_num_frac_points.count(0)}')

---------- Print checks for the input pc blocks ----------
The input point cloud was divided into 121 blocks.
The number of points of the original input pc that are now contained within blocks is: 136658
The number of blocks that have more points than the subsampling goal: 1
The number of blocks that have less points then the subsampling goal: 120
The number of blocks which have no subsampled points at all: 39
The maximum amount of points within a block is: 4328
The average amount of points of the blocks that have enough points is: 4328.0
The average amount of points of the blocks that have to few points is: 1102.75
On average there are 14.19 fracture points within a block.
On average the blocks with enough points have 58.0 fracture points within a block.
On average the blocks with fewer points have 13.82 fracture points within a block.
The amount of blocks that do not contain any fracture points are: 62


## Visualization of input PC and enlarged block center coordinates

Description coming...

In [20]:
# Path to input data
input_data_path = ROOT + "/data/testdata/data_labelled_int.npy"

# Load the input data
input_data = np.load(input_data_path)

# Assigning the coordinates
input_points = np.stack([input_data[:, 0], 
                         input_data[:, 1], 
                         input_data[:, 2]], axis=1)
# Assigning color channels
red_c   = np.array(input_data[:, 3])
green_c = np.array(input_data[:, 4])
blue_c  = np.array(input_data[:, 5])
# Converting color channels to [0, 1] range
red_c   = (red_c - np.min(red_c)) / (np.max(red_c) - np.min(red_c))
green_c = (green_c - np.min(green_c)) / (np.max(green_c) - np.min(green_c))
blue_c  = (blue_c - np.min(blue_c)) / (np.max(blue_c) - np.min(blue_c))
# Creating color data
color_data = np.stack([red_c, green_c, blue_c], axis=1)

# Appending the block center coordinates to the visualisation data
input_points = np.vstack((input_points, blocks_center_coords))
block_centers_red_c = np.ones(len(blocks_center_coords))
block_centers_green_c = np.zeros(len(blocks_center_coords))
block_centers_blue_c = np.zeros(len(blocks_center_coords))
block_center_colors = np.stack([block_centers_red_c, block_centers_green_c, block_centers_blue_c], 
                               axis=1)
color_data = np.vstack((color_data, block_center_colors))

# Make the point cloud to be displayed
viz_pcd = makePC(input_points, color_data)

# Print check for point cloud
print("------ Print check for input point cloud ------")
print(f"The input point cloud consists of {len(viz_pcd.points)} number of points.")
print(f"The minimum coordinates of the point cloud are: {np.amin(np.asarray(viz_pcd.points), axis=0)}")
print(f"The maximum coordinates of the point cloud are: {np.amax(np.asarray(viz_pcd.points), axis=0)}")
# Minimum will be [0, 0, 0] as the input data has been shifted already

# Cropping the point cloud using bounding boxes
bbox = viz_pcd.get_oriented_bounding_box()
bbox.color = (0, 1, 0) #bbox in green
viz_pcd_cropped = viz_pcd.crop(bbox)

# Print check for cropped point cloud
print("------ Print check for cropped input point cloud ------")
print(f"The cropped point cloud consists of {len(viz_pcd_cropped.points)} number of points.")
print(f"The bounding box cropping removed {len(viz_pcd.points) - len(viz_pcd_cropped.points)} points.")
print(f"The minimum coordinates of the cropped point cloud are: {np.amin(np.asarray(viz_pcd_cropped.points), axis=0)}")
print(f"The maximum coordinates of the point cloud are: {np.amax(np.asarray(viz_pcd_cropped.points), axis=0)}")
# Minimum will be [0, 0, 0] as the input data has been shifted already

# Visualizing the point cloud
o3d.visualization.draw([viz_pcd_cropped])

------ Print check for input point cloud ------
The input point cloud consists of 662646 number of points.
The minimum coordinates of the point cloud are: [0. 0. 0.]
The maximum coordinates of the point cloud are: [288.875869 226.199528 177.267736]
------ Print check for cropped input point cloud ------
The cropped point cloud consists of 662645 number of points.
The bounding box cropping removed 1 points.
The minimum coordinates of the cropped point cloud are: [0. 0. 0.]
The maximum coordinates of the point cloud are: [288.875869 226.199528 177.267736]


[error] GLFW error: Cocoa: Failed to find service port for display


: 